## Community Erkennung 

Louvain: um hierarchische Community-Strukturen zu entdecken und dabei die Modularität zu maximieren. Er ist effizient in grossen Netzwerken <br>
Label Proagation: schneller und einfacher als Louvain und basiert auf der Idee, dass Knoten die Labels ihrer Nachbarn übernehmen. Er eignet sich besonders gut für Netzwerke, in denen die Community-Struktur dynamisch ist oder wo eine schnelle, vorläufige Einschätzung der Community-Struktur benötigt wird.

#### Cypher Louvain Query

In [ ]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", ""))

louvain_query = """
CALL gds.louvain.stream('myGraphConnections')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).name AS name, communityId
ORDER BY communityId
"""
louvain_results = graph.run(louvain_query).data()

for result in louvain_results:
    print(result)


Output Deutung:<br>
Community ID = Gemeinschaftsidentifikation

#### Übergabe an GPT

In [ ]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=150):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message['content'].strip() 
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [ ]:
openai_wrapper = OpenAIWrapper(api_key='')
prompt_intro = "Based on the Louvain Community Detection algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:\n\n"
prompt_body = "\n".join([f"- {res['name']} in community {res['communityId']}" for res in louvain_results[:10]])  # Top 10 
prompt_topic = "\n\nGiven that these communities may represent different clusters of entities within the network, it's important to analyze if any of these patterns indicate potential money laundering activities. The identification of suspicious patterns and the subsequent decision on whether they constitute money laundering activities require careful consideration of various factors, including transaction patterns, individuals involved, and community structures."
prompt_question = "\n\nWhat patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")


#### Cypher Proagation Query

In [ ]:
label_propagation_query = """
CALL gds.labelPropagation.stream('myGraphConnections')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).name AS name, communityId
ORDER BY communityId
"""
label_propagation_results = graph.run(label_propagation_query).data()

for result in label_propagation_results:
    print(result)


Output Deutung: <br>
Community ID = Gemeiscnaftszuordnung

#### Übergabe an GPT

In [ ]:
prompt_intro = "Based on the label_propagation algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:\n\n"
prompt_body = "\n".join([f"- {res['name']} in community {res['communityId']}" for res in label_propagation_results[:10]])  # Top 10 
prompt_topic = "\n\nGiven that these communities may represent different clusters of entities within the network, it's important to analyze if any of these patterns indicate potential money laundering activities. The identification of suspicious patterns and the subsequent decision on whether they constitute money laundering activities require careful consideration of various factors, including transaction patterns, individuals involved, and community structures."
prompt_question = "\n\nWhat patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")